In [2]:
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2

%load_ext autoreload

In [3]:
sys.path.append("../")



In [4]:
os.listdir("..\data")
path = "..\data"
targets = []
for folder in os.listdir(path):
    for f in os.listdir(os.path.join(path, folder)):
        if f.endswith(".png") or f.endswith(".jpg"):
            targets.append(os.path.join(path, folder, f))
print(targets)

['..\\data\\clockwork_drake\\clockwork_drake.jpg', '..\\data\\crelvohr\\crelvohr.png', '..\\data\\hermit_1\\hermit_1.png', '..\\data\\hermit_2\\hermit_2.png', '..\\data\\madc_arcanamite\\Monster a Day Compendium V2-004.png', '..\\data\\madc_crystalline_dragon\\Monster a Day Compendium V2-016.png', '..\\data\\madc_kobolds\\Monster a Day Compendium V2-048.png', '..\\data\\madc_seryms\\Monster a Day Compendium V2-067.png', '..\\data\\oc_artificers\\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-05.png', '..\\data\\oc_artillerist\\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-14.png', '..\\data\\oc_berserker\\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-17.png', '..\\data\\oc_cleric\\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-22.png', '..\\data\\priest_and_scout\\priest_and_scout.png', '..\\data\\rak_tulkesh\\rak_tulkesh.png', '..\\data\\The Lord of the Deep\\The Lord of the Deep.png']


In [14]:
%autoreload 2

from data_loaders.textract_image_loader import TextractImageLoader
from extractor import StatblockExtractor
from preprocessing.columniser import Columniser
from utils.config import get_config, get_argparser
from utils.logger import get_logger

from fifthedition.creature_printer import pretty_format_creature
from fifthedition.creature_parser import CreatureParser
from fifthedition.creature_schema import CreatureSchema



# Get arguments
parser = get_argparser()
args = parser.parse_args(["..\data\clockwork_drake\clockwork_drake.jpg", "--config", r"..\default.conf", "--cache",  r"..\.cache" ])
print(args)

# Get config file
config = get_config(args.config)

logger = get_logger(args.debug, args.notebook, args.logs)

Namespace(targets=['..\\data\\clockwork_drake\\clockwork_drake.jpg'], source=None, output=None, cache='..\\.cache', config='..\\default.conf', logs=None, debug=False, notebook=False)


In [16]:
#fig = plt.figure(figsize=(20, 20))

try:
    config.add_section("clusterer")
except:
    pass
config.set("clusterer", "fuzzyness", '3')

for target in targets[9:10]:
    print(target)
# target = targets[]
    
    se = StatblockExtractor(config, logger)
    se.register_data_loader(TextractImageLoader)
    #print(target)
    se.load_data(target)
    #x = se.parse(draw_statblocks=True, draw_lines=True, draw_columns=True, draw_final_columns=False, draw_clusters=True)
    x = se.parse()

    for page in x:
        logger.info("Page {}:".format(page))
        for statblock in x[page]:
            cp = CreatureParser(config, logger)
            creature = cp.statblock_to_creature(statblock)
            print()
            print(pretty_format_creature(creature))
            print()
            try:
                CreatureSchema.validate(creature.data)
            except Exception as e:
                print("Failed to validate")
                print(e)

..\data\oc_artillerist\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-14.png
[2021-07-03 10:35:20,389] INFO     sbp.txloader     Retrieving cached result for ..\data\oc_artillerist\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-14.png
[2021-07-03 10:35:20,410] INFO     sbp              Loaded file ..\data\oc_artillerist\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-14.png
[2021-07-03 10:35:20,411] INFO     sbp              Loading ..\data\oc_artillerist\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses-14.png
[2021-07-03 10:35:20,415] INFO     sbp              Page 0:

Trebuchet
Huge construct, 
-------------------------------------------------------
Armor Class 13
Hit Points 40
-------------------------------------------------------

-------------------------------------------------------

-------------------------------------------------------


Failed to validate
Missing keys: 'abilities', 'speed'

Trebucheter                                

In [20]:
for l in x[0][0].lines:
    print(l.text, l.attributes)


CLOCKWORK DRAKE ['statblock_title']
Huge, Unaligned ['race_type_header']
Armor Class 18 ['ac']
Hit Points 225 (18d12 + 108) ['dice_roll', 'hp']
Speed 40 ft., fly 80 ft. ['speed', 'block_title']
STR DEX CON INT WIS CHA ['array_title']
25 (+7) 8 (-1) 23 (+6) 12 (+1) 15 (+2) 8 (-1) ['array_values']
Saving Throws Dex +4, Con + 11, Wis +7, Cha +4 ['saves']
Skills Arcana +6, Athletics + 12, Perception +7 ['skills']
Damage Resistances Fire, Lightning; Bludgeoning, ['resistances']
Piercing, and Slashing from Nonmagical Attacks that []
aren't Adamantine []
Damage Immunities Poison, Psychic ['dam_immunities']
Condition Immunities Charmed, Exhaustion, Frightened, ['con_immunities']
Paralyzed, Petrified, Poisoned []
Senses Blindsight 60 feet, Darkvision 1250 ft., Passive ['senses']
Perception 22 []
Languages Understands the languages of its creator but ['languages']
can't speak []
Challenge 16 (15,000 XP) ['cr']
Proficiency Bonus: +5 ['proficiency']
Adamantine Weapons. The Clockwork Drake's weapon

In [146]:
import schema as old_schema
old_schema.Schema(CreatureSchema.schema["ac"]).validate(creature.data["ac"])

[{'ac': 18, 'from': ['natural armor'], 'condition': ''}]

In [740]:
import creature
importlib.reload(creature)

def set_features(char, line, attribs):
    in_feature = False
    if "senses" in attribs:
        char.set_senses(line)
        in_feature=True
    if "cr" in attribs:
        char.set_cr(line)
        in_feature=True
    if "skills" in attribs:
        char.set_skills(line)
        in_feature=True
    if "resistances" in attribs:
        char.set_resistances(line)
        in_feature=True
    if "vulnerabilities" in attribs:
        char.set_vulnerabilities(line)
        in_feature=True
    if "languages" in attribs:
        char.set_languages(line)
        in_feature=True
    if "saves" in attribs:
        char.set_saves(line)
        in_feature=True
    if "dam_immunities" in attribs:
        char.set_immunities(line)
        in_feature=True
        pass
    if "con_immunities" in attribs:
        char.set_condition_immunities(line)
        in_feature=True
        pass
    if "vulnerabilities" in attribs:
        char.set_vulnerabilities(line)
        in_feature=True
        pass
    
    if in_feature:
        return True
    else:
        return False

def parse_defence_block(char, current_block, line, bound, attrib):

    block_finished = False
    state = 3

    #Check if the next line is offset from the current, but still within the same column
    is_offset = len(current_block) > 0 and bound["left"] > current_block[2]["left"] + 5 and current_block[2]["left"] + current_block[2]["width"] > bound["left"]

    next_current_block = [attrib, [line.strip()], bound]

    if len(current_block) > 0  and ("cr" in current_block[0] or "proficiency" in current_block[0]):
        block_finished = True
        next_current_block = []
        state = 4
    elif is_offset or len(attrib) == 0:
        if current_block == []:
            return [[], [line.strip()], bound], 3
        else:
            current_block[1].append(line)
            return current_block, 3
    else:
        if len(current_block) > 0:
            if len(attrib) > 0:
                block_finished = True

    in_feature=False
    if block_finished:
        l = " ".join(current_block[1])
        a = current_block[0]
        if not set_features(char, l, a):
            state = 4


    return next_current_block, state


def parse_feature_block(char, current_block, line, bound, attrib):

    parts = line.split(".")
    tag = parts[0]

    if len(parts) == 2:
        text = parts[1]
    else:
        text = ".".join(parts[1:])

    new_block = len(current_block) == 0 #Easy case, new block
    #Check first sentence is less than 6 words (not including anything in brackets) and starts with a capital
    has_title = len(line.split()) > 5 and len(tag.split("(")[0].split()) < 6 and tag[0].isupper() and tag.split()[0].lower() not in constants.ABILITIES
    is_end = "end" in attrib
    
    #Does the line finish early (aka end of paragraph). Ignore this for spell blocks
    is_short = len(current_block) > 0 and (bound["width"] < current_block[2]["width"] * 0.8) and "spellcasting" not in current_block[0].lower()

    spell_list = False
    if len(current_block) > 0 and "spellcasting" in current_block[0].lower():
        if re.match("(cantrip|1st|2nd|3rd|[4-9]th)", line, re.IGNORECASE):
            spell_list = True

    #Check if we're at the start of a new feature
    if not spell_list and (new_block or has_title or is_end):
        if len(current_block) > 0:
            char.add_feature(current_block[0], current_block[1])
            current_block = []
        current_block = [tag, [text.strip()], bound]
    else:
        current_block[1].append(line.strip())
        if is_short:
            char.add_feature(current_block[0], current_block[1])
            current_block = []

    return current_block

def parse_action_block(char, current_block, line, bound, attrib):
    parts = line.split(".")
    tag = parts[0]
    text = ".".join(parts[min(1, len(parts)):])

    new_block = len(current_block) == 0 #Easy case, new block
    #Check first sentence is less than 6 words (not including anything in brackets) and starts with a capital
    has_title = len(line.split()) > 5 and len(tag.split("(")[0].split()) < 6 and tag[0].isupper() and tag.split()[0].lower() not in constants.ABILITIES
    #Does the line finish early (aka end of paragraph). Ignore this for spell blocks
    is_short = False#len(current_block) > 0 and (bound["width"] < current_block[2]["width"] * 0.75)
    is_attack_start = "melee_attack" in attrib or "ranged_attack" in attrib
    is_end = "end" in attrib
    is_table = len(line) > 0 and line[0].isnumeric()

    #Check if we're at the start of a new feature
    if not is_table and (new_block or has_title or is_attack_start or is_end):
        if len(current_block) > 0:
            char.add_action(current_block[0], current_block[1])
            current_block = []
        current_block = [tag, [text.strip()], bound]
    else:
        if is_table:
            current_block[1].append("\n" + line.strip())
        else:
            current_block[1].append(line.strip())
        # if is_short:
        #     char.add_action(current_block[0], current_block[1])
        #     current_block = []

    return current_block

def parse_legendary_action_block(char, current_block, line, bound, attrib):
    parts = line.split(".")
    tag = parts[0]
    text = ".".join(parts[min(1, len(parts)):])

    new_block = len(current_block) == 0 #Easy case, new block
    #Check first sentence is less than 6 words (not including anything in brackets) and starts with a capital
    has_title = len(line.split()) > 5 and len(tag.split("(")[0].split()) < 6 and tag[0].isupper() 
    #Does the line finish early (aka end of paragraph). Ignore this for spell blocks
    is_short = False#len(current_block) > 0 and (bound["width"] < current_block[2]["width"] * 0.75)
    is_end = "end" in attrib
    is_table = len(line) > 0 and line[0].isnumeric()

    #Check if we're at the start of a new feature
    if not is_table and (new_block or has_title or is_end):
        if len(current_block) > 0:
            char.add_legendary_action(current_block[0], current_block[1])
            current_block = []
        current_block = [tag, [text.strip()], bound]
    else:
        if is_table:
            current_block[1].append("\n" + line.strip())
        else:
            current_block[1].append(line.strip())
        # if is_short:
        #     char.add_action(current_block[0], current_block[1])
        #     current_block = []

    return current_block


def parse_statblock(lines):

    char = creature.Creature()

    states = [
        0, #title,
        1, #defence
        2, #attributes
        3, #flavour
        4, #features
        5, #actions
        6, #legendary actions
    ]

    attrib_vals = ["array_values", "array_title"]
    flavour_vals = ["senses", "cr", "languages", "skills", "resistances", "saves"]

    current_block = []
    state = 0

    for i in range(len(lines)):
        #print(lines[i])
        line,bound,attrib = lines[i]
        #line = line.replace("\n", " ")

        print(line)
        print(attrib)
        #print()

        ### Look ahead to join lines
        if state < 3 and i+2 < len(lines):
            while len(lines[i+1][2]) == 0:
                line += " " + lines[i+1][0]
                i += 1
                print(i, lines[i+1])

        #print(line, attrib)

        ### Force Switch to Later States:
        if state < 2:
            for a in attrib: 
                if a in attrib_vals:
                    print("WARNING. Did not find complete defence block")
                    state = 2
                    break

        if state < 3:
            for a in attrib:
                if a in flavour_vals:
                    print("WARNING. Did not find attributes")
                    state = 2
                    break

        if state < 5:
            if "action_title" in attrib:
                state = 5
                if len(current_block) > 0:
                    char.add_feature(current_block[0], current_block[1])
                    current_block = []
                continue

        if state < 6:
            if "legendary_action_title" in attrib:
                state = 6
                if len(current_block) > 0:
                    char.add_action(current_block[0], current_block[1])
                    current_block = []
                continue

        ### Header Block
        if state == 0 and "statblock_title" in attrib:
            char.set_name(line)
        elif state == 0 and "race_type_header" in attrib:
            char.set_size_type_alignment(line)
            state = 1

        ### Defense Block
        if state == 1:
            if "hp" in attrib:
                char.set_hp(line)
            if "speed" in attrib:
                char.set_speed(line)
            if "ac" in attrib:
                char.set_ac(line)

            if len(char.hp) > 0 and len(char.speed) > 0 and len(char.ac) > 0:
                state = 2
                continue

        ### Attributes
        if state == 2:
            if "array_values" in attrib:
                char.set_attributes(line)
                state = 3
                continue

        ### Flavour
        if state == 3:
            current_block, state = parse_defence_block(char, current_block, line, bound, attrib)
            if state > 3 and len(current_block) > 0:
                line = " ".join(current_block[1])
                bound = current_block[2]
                attrib = current_block[0]
                current_block = []

        ### Features
        if state == 4:
            current_block = parse_feature_block(char, current_block, line, bound, attrib)

        ### Actions
        if state == 5:
            current_block = parse_action_block(char, current_block, line, bound, attrib)

        ### Legendary Actions
        if state == 6:
            current_block = parse_legendary_action_block(char, current_block, line, bound, attrib)



    #for a in char.to_json()["action"]:
    #    print(a)  
    return char



In [786]:
import json
for target in ["priest_and_scout"]:#statblocks:
    for i,sb in enumerate(statblocks[target]):
        print("Parsing {}-{}".format(target, i))
        char = parse_statblock(sb[0])
        print("=========================================================")        
        char.print()
        print("=========================================================\n")
        with open("statblocks/{}_{}.json".format(target, char.name), 'w') as f:
            json.dump(char.to_json(), f)

Parsing priest_and_scout-0
PRIEST Spellcasting. The priest is a 5th-level spellcaster. Its spellcasting
['block_title', 'statblock_title']
Medium humanoid (any race), any alignment ability is Wisdom (spell save DC 13, +5 to hit with spell attacks).
['race_type_header']
2 ['Armor Class 13 (chain shirt)', {'left': 368.06780913472176, 'top': 192.84490431845188, 'width': 151.14360563457012, 'height': 15.172144621610641}, ['ac']]
The priest has the following cleric spells prepared:
[]
Armor Class 13 (chain shirt)
['ac']
4 ['Hit Points 27 (5d8 + 5)', {'left': 368.28449353575706, 'top': 218.83252781629562, 'width': 122.06635929644108, 'height': 14.742284074425697}, ['dice_roll', 'hp']]
Cantrips (at will): light, sacred flame, thaumaturgy
[]
Hit Points 27 (5d8 + 5)
['dice_roll', 'hp']
6 ['Speed 30 ft.', {'left': 368.65033373236656, 'top': 244.9408999979496, 'width': 64.38336034491658, 'height': 14.625553973950446}, ['speed', 'block_title']]
1st level (4 slots): cure wounds, guiding bolt, sanct

IndexError: list index out of range

In [681]:
target = "hermit_2"


In [734]:
statblocks = []
se = builders.StatblockExtractor(stl, col, la, clus, ca, sb)
statblocks = se.parse()
statblocks[target][0]

Loading clockwork_drake


AttributeError: 'list' object has no attribute 'create_statblocks'